## Практическое задание

<ol>
    <li>На выбор: либо обучить сверточную нейронную сеть на любом датасете, который не использовался на уроке, либо реализовать нейронную сеть AlexNet на любом датасете.
    </li>
</ol>

Я нашёл статью, в которой AlexNet была представлена, и взял оттуда архитектуру (изменив размер ядер). В статье архитектура такая (числа в скобках - это характеристики слоя, для Convolutional слоёв это размер стороны ядра и размерность выхода, для пулинга - размер ядра, для полносвязных - количество нейронов, так же я использовал обозначение {}\*2 - это 2 раза повторяется то, что в скобках, чтобы на одну строку влезло):

`conv(11,96)-maxpool(3)-conv(5,256)-maxpool(3)-{conv(3,384)-}\*2-conv(3,256)-maxpool(3)-dense(4096)-dense(4096)-dense(1000)`

Результаты на тесте с вариациями:

`0.3671: conv(3,32)-maxpool(2)-conv(5,32)-maxpool(2)-{conv(3,64)}*3-maxpool(2)-dense(512)-dense(512)-dense(10)`<br>
`0.3169: conv(3,32)-maxpool(2)-conv(5,32)-maxpool(2)-{conv(3,64)}*3-maxpool(2)-dense(40) -dense(40) -dense(10)`<br>
`0.2367: conv(3,32)-maxpool(2)-conv(5,32)-maxpool(2)-{conv(3,64)}*2-conv(3,40)-maxpool(2)-dense(40) -dense(40) -dense(10)`<br>
`0.3685: conv(3,32)-maxpool(2)-conv(5,32)-maxpool(2)-{conv(3,64)}*2-conv(3,40)-maxpool(2)-dense(512)-dense(512)-dense(10)`<br>
`0.3495: conv(5,32)-maxpool(2)-conv(5,32)-maxpool(2)-{conv(3,64)}*2-conv(3,40)-maxpool(2)-dense(512)-dense(512)-dense(10)`<br>
`0.3884: conv(5,32)-maxpool(2)-conv(5,32)-maxpool(2)- conv(3,64)   -conv(3,40)-maxpool(2)-dense(512)-dense(512)-dense(10)`<br>
`0.3936: conv(5,32)-maxpool(2)-conv(5,32)-maxpool(2)- conv(3,64)   -conv(3,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.3969: conv(3,32)-maxpool(3)-conv(5,32)-maxpool(2)- conv(3,64)   -conv(3,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.3502: conv(3,32)-maxpool(3)-conv(5,32)-maxpool(3)- conv(3,64)   -conv(3,40)-maxpool(3)-dense(1024)-dense(1024)-dense(10)`<br>
`0.4210: conv(5,32)-maxpool(5)-conv(3,32)-maxpool(2)- conv(3,64)   -conv(3,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.3498: conv(5,32)-maxpool(5)-conv(3,32)-maxpool(2)-conv(3,64)-conv(2,64)-conv(2,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.3633: conv(5,32)-conv(3,32)-maxpool(5)-conv(3,32)-maxpool(2)-conv(3,64)-conv(2,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.4492: conv(5,32)-conv(3,32)-conv(3,32)-maxpool(2)-conv(3,64)-conv(2,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.4630: conv(5,32)-conv(3,32)-conv(3,32)-conv(3,64)-conv(2,40)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>
`0.3696: conv(5,32)-maxpool(5)-dense(1024)-conv(3,32)-conv(3,64)-maxpool(2)-dense(1024)-dense(1024)-dense(10)`<br>

In [39]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

from math import ceil


# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


n_points = len(x_train)
steps_per_epoch = ceil(n_points / batch_size)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (5, 5), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=2))
model.add(Dense(1024))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(40, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=steps_per_epoch)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры
Использование data augmentation в реальном времени
Epoch 1/1
1563/1563 [==============================] - 584s 373ms/step - loss: 1.8459 - acc: 0.3177 - val_loss: 1.7622 - val_acc: 0.3696
сохранить обученную модель как C:\Users\Viktor\Documents\Python\NN\saved_models\keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 25s 3ms/step
Test loss: 1.7622144187927247
Test accuracy: 0.3696


## Результат

1) Увеличение ядра свёртки или уменьшение размерности выхода свёрточного слоя снижает точность. <br>
2) MaxPooling слои ухудшают результат, но очень сильно увеличивают скорость обучения.<br>
3) Количество нейронов в полносвязных слоях сильно увеличивают продолжительность обучения, а вот результат улучшают немного. Тоже самое при добавлении новых полносвязных слоёв в конце. Если полносвязный добавлен в середину, т.е. до пуллинга, то параметров будет в нём много и такой слой будет очень долго обучается.